In [202]:
import pandas as pd

# Optimal Play in Oh Hell
## (or how to use python to ruin game night)

In case you aren't familiar, oh hell is a trick taking game a little bit like hearts. The major difference is that at the start of each round, each participant guesses how many tricks they will take. You only get points if you get *exactly* that many tricks. Another peculiarity of the game is that the size of the hand changes each round, starting at ten and going down by one till you are dealt just a single card. 

Lately, I’ve been losing at Oh Hell. A lot. There’s a fair bit of luck involved in the game, but especially in the round with one card I figured that there must be a “right” answer to what you should bid. 

For example, Let's assume you've dealt (which means you're leading) You have a binary decision to make- you can guess one, saying you will take the trick, or 0 saying that you won't take the trick. You have three pieces of information- the card in your hand, how many other players there are, and one card you know is out of the game (one card is flipped over to select the trump at the start of each round). Let’s assume you have a 9 of spades, and you’ve flipped over a king of hearts to choose the trump. 

To beat your card, another player would have to have a card of the same suit bigger than your card or a trump or a higher trump if the card in your hand is a trump. We can boil the question down to something more tractable- what is the probability that someone else at the table has a card that will beat your card? You should bet 1 if there is a more than 50% chance you will win the hand, otherwise you should bet 0. 

Okay, so based on all the trick taking rules we know two full suites won't take your card. Spades and clubs won't take the trick, and neither will the 2 of diamonds through the 8 of diamonds. That's a total of 33 cards (26 of the other suites, 7 of the lower cards in the same suite) that won't take the trick. There are 50 cards in the deck, and we dealt 2 of them to the other players. 

Well, for the first player we know they have a 33/50 chance of not picking a card that's bigger than our card. Assuming that's happened, the second player has a 32/49 chance of not picking a card that's bigger than our card. Multiplying those probabilities we get:

In [220]:
round(33/50 * 32/49, 3)

0.431

So you should guess 0 in this scenario!

Let's see if we can generalize this solution. If we can figure out what the lowest card you should bet 1 on is for each number of players, that could be the competitive advantage I'm looking for &#x1F600;

From our last example, we can see we need to know how many cards won't beat our hand. It's a little easier to calculate how many cards would beat our card, and we can code that up quickly here.

In [221]:
# I'm going to work with a numbering system for cards from 2-28 so that the non-trump number cards match their value.
# Facecards count up in value from 10, so jack = 11 ... ace = 14 etc. The lowest trump will be 15 and so on. 
# All the non-trump suites will have an equal probability, so we don't need a seperate input for suite

def get_n_takes(tcard_out_val, in_hand_val):
    
    n_takes = 27 - in_hand_val

    # If the trump card out would beat our card, we subtract 1 from the possible cards that would take
    if tcard_out_val > in_hand_val:
        n_takes = n_takes - 1
    
    return n_takes

Next, we need to use that information to get the probability each player draws a card that doesn't take the trick. Extending the above example, we can see that for each additional individual in the game, we need to multiply the probability by the number of cards that won't take / the total number of cards. 

These events are dependent- each card dealt that doesn't take means one less card that won't take, and one less card overall. Formally, let's say the number of other players is n, and the number of cards that won't take the trick are c. We can say the probability of us taking the trick as the leader is: $$ \prod_{i=1}^n \frac{c - i}{50 - i}$$

That's pretty easy to code with our `get_n_takes` function above!

In [222]:
def prob_win_trick(tcard_out_val, in_hand_val, num_players):
    
    num_cards_that_take = get_n_takes(tcard_out_val, in_hand_val)
    probability_of_taking = 1
    
    for n in range(num_players - 1): # We subtract one here to get the # of other players
        prob_no_take = ((50 - num_cards_that_take - n) / (50 - n))
        probability_of_taking = probability_of_taking * prob_no_take
        
    return probability_of_taking

Let's put that all together to look at the probability of taking the trick for any card, each trump card value, and number of players. Let's go check out that feature space!

In [223]:
values = range(2, 28) 
players = range(2, 6) # Only 
trump_card_vals = range(15, 28)
results = []

for player_num in players:
    for your_card_val in values:
        for trump_card_val in trump_card_vals:
            prob_take = prob_win_trick(trump_card_val, your_card_val, player_num)
            results.append( {
                'probability_take' : prob_take,
                'card_value' : your_card_val,
                'trump_card_val' : trump_card_val,
                'players' : player_num
            })

In [224]:
prob_win_df = pd.DataFrame(results)
prob_win_df = prob_win_df[~(prob_win_df['trump_card_val'] == prob_win_df['card_value'])]

In [225]:
prob_win_df

,probability_take,card_value,trump_card_val,players
0,0.52,2,15,2
1,0.52,2,16,2
2,0.52,2,17,2
3,0.52,2,18,2
4,0.52,2,19,2
...,...,...,...,...
1346,1.00,27,22,5
1347,1.00,27,23,5
1348,1.00,27,24,5
1349,1.00,27,25,5


Sucsess! We've got a dataframe of all the probabilities. Thanks Python. 

Now we just have to find the minimum card value for each number of players. There's a little wrinkle here- for more than 3 players, there are actually two lowest cards depending on what we know the trump card is! That makes getting our inflection point a little tricky, but since we'd expect the probabilities to remain the same when the trump card we can filter for the cards with those probabilities and their lowest trump.

In [249]:
prob_win_df['should_take'] = prob_win_df['probability_take'] > .5
only_takes = prob_win_df[prob_win_df['should_take']].sort_values(['probability_take', 'card_value'])
min_prob = only_takes.loc[only_takes.groupby(['players'])['probability_take'].idxmin().dropna()][['probability_take', 'players']].rename({'probability_take' : 'min_probability_take'}, axis = 1)
only_takes = pd.merge(only_takes, min_prob) 
lowest_prob = only_takes[only_takes['probability_take'] == only_takes['min_probability_take']].drop('min_probability_take', axis = 1)

In [250]:
lowest_card_bet_one = lowest_prob.groupby(['players', 'card_value']).max('trump_card_val').reset_index()

Next, we can make everything nice and human readable

In [251]:
human_names_card_vals = {}
face_cards = ['jack', 'queen', 'king', 'ace']
for card_val in range(1,28):
    if card_val < 11:
        human_names_card_vals[card_val] = f'{card_val}'
    if card_val >= 11 and card_val < 15:
        human_names_card_vals[card_val] = f'{face_cards[card_val - 11]}'
    if card_val >= 15 and card_val < 24:
        human_names_card_vals[card_val] = f'trump {card_val - 13}'
    if card_val >= 24:
        human_names_card_vals[card_val] = f'trump {face_cards[card_val - 24]}'

In [252]:
lowest_card_bet_one['trump_card_val'] = lowest_card_bet_one['trump_card_val'].map(human_names_card_vals)
lowest_card_bet_one['card_value'] = lowest_card_bet_one['card_value'].map(human_names_card_vals)

In [253]:
lowest_card_bet_one['cumcount_vals'] = (lowest_card_bet_one.groupby('players').cumcount() + 1).astype(str)

In [254]:
lowest_card_bet_one = lowest_card_bet_one.pivot(index = 'players', columns = 'cumcount_vals').reset_index()

In [255]:
lowest_card_bet_one.columns = ['_'.join(col).strip('_') for col in lowest_card_bet_one.columns.values]

In [256]:
lowest_card_bet_one

,players,card_value_1,card_value_2,probability_take_1,probability_take_2,trump_card_val_1,trump_card_val_2,should_take_1,should_take_2
0,2,2,NaN,0.520000,NaN,trump ace,NaN,True,NaN
1,3,queen,NaN,0.514286,NaN,trump ace,NaN,True,NaN
2,4,trump 3,trump 4,0.504082,0.504082,trump ace,trump 3,True,True
3,5,trump 6,trump 7,0.535866,0.535866,trump ace,trump 6,True,True


In [236]:
transition_table = lowest_card_bet_one[['players', 'card_value_1', 'card_value_2', 'trump_card_val_1']]

In [242]:
transition_table.rename({'card_value_1' : 'below_the_trump',
                         'card_value_2' : 'above_the_trump',
                        'trump_card_val_1' : 'trump'}, axis = 1)

,players,below_the_trump,above_the_trump,trump
0,2,2,NaN,trump 2
1,3,queen,NaN,trump 2
2,4,trump 3,trump 4,trump 4
3,5,trump 6,trump 7,trump 7


In [248]:
only_takes[only_takes['players'] == 4][0:25]

,probability_take,card_value,trump_card_val,players,should_take,min_probability_take
0,0.504082,16,17,4,True,0.504082
1,0.504082,16,18,4,True,0.504082
2,0.504082,16,19,4,True,0.504082
3,0.504082,16,20,4,True,0.504082
4,0.504082,16,21,4,True,0.504082
5,0.504082,16,22,4,True,0.504082
6,0.504082,16,23,4,True,0.504082
7,0.504082,16,24,4,True,0.504082
8,0.504082,16,25,4,True,0.504082
9,0.504082,16,26,4,True,0.504082
